# Problemas das N-Rainhas

**Professor:**
Cristiano Leite de Castro

**Alunos:**
Gabriel Camatta Zanotelli	2018020140
Lucas de Almeida Martins	2018020328

## Introdução

O problema proposto neste trabalho consiste no desenvolvimento de um algoritmo com base em computação evolucionária, que promova a disposição de N rainhas em um tabuleiro de xadrez de tamanho também N, de forma que a quantidade de xeques entre as rainhas seja o menor possível.


## Implementação

O código foi desenvolvido na linguagem python e o problema foi modelado de como uma permutação de números inteiros de N elementos, de forma que cada posição da permutação representa uma coluna do tabuleiro e o valor de cada posição representa a linha da respectiva rainha na coluna. Essa representação foi escolhida, pois ela gera uma população inicial sem a possibilidade de rainhas sofrerem xeques na horizontal e na vertical.
Além disso, serão analisadas a seguir a implementação de cada parte do código em sua ordem de execução.


### Definições de variáveis iniciais

In [59]:
import random as rd
import numpy as np
import matplotlib.pyplot as plt

In [60]:
crossover_rate = 1
mutation_rate = 0.8

base_population = 20
pop_sample = 5

fitness_history = []


### Funções de suporte

In [70]:
def argsort(seq):
    return sorted(range(len(seq)), key=seq.__getitem__)

def verify_condition(pupulation):
    return 0 in pupulation

def get_best_solution(population_fit):
    count = 0
    aux = 100
    for i in range(len(population_fit)):
        if population_fit[i] < aux:
            aux = population_fit[i]
            count = i
    return i

def draw_pop(pop):
    for i in range(len(pop)):
        line = ''.format((8-i), ' ')
        for j in range(len(pop)):
            if pop[j] == i:
                line = line + "X "
            else:
                line = line + "- "
        print(line)


### Definição da População Inicial

Utilizamos uma função específica que recebe dois argumentos, a população de cada geração e o tamanho N do tabuleiro (que consequentemente é a quantidade de rainhas). Os parâmetros podem ser passados como argumentos ou a função pode ser chamada utilizando valores padrão.

A função utiliza da biblioteca de random do Python e retorna um vetor vetor de vetores, em que cada campo é uma solução possível do resultado. É utilizado um método específico da biblioteca de forma que nenhuma rainha esteja na mesma linha na geração inicial, já minimizando ao máximo o número de cheques.

In [62]:
def init_population(_mu:int = 20, n:int = 8):
    population = []
    for i in range (_mu):
        population.append(rd.sample(range(n), n))
    return population

### Avaliação de candidatos

Utilizamos de um método já bem difundido de análise de matrizes para checar cada tabuleiro individualmente, retornando um inteiro que representa o número de cheques no tabuleiro. Vale notar que o número será sempre par, uma vez que uma rainha que coloca outra em cheque, ficará, consequentemente, em cheque também.

In [63]:
def fitness_nq(solution):
    xeques = 0
    for i in range(0,len(solution)):
        for j in range(0,len(solution)):
            if i!=j:
                if i-solution[i] == j-solution[j] or i+solution[i] == j+solution[j]:
                    xeques+=1
    return xeques

## Seleciona pais

Para a seleção dos pais passamos uma população inteira, assim como uma “amostra de população”, no caso dos testes feitos, o valor utilizado foi 5.

Um número de soluções aleatórias igual ao tamanho da amostra são selecionadas e avaliadas. Os dois mais bem adaptados serão os utilizados para recombinação.


In [64]:
def select_parents(pupulation, pop_sample):

    sample_pop = rd.sample(pupulation, pop_sample)

    sample_pop_fit = [0] * pop_sample
    for i in range(len(sample_pop_fit)):
        sample_pop_fit[i] = fitness_nq(sample_pop[i])

    parents_id = np.argsort(sample_pop_fit)[:2]
    parents = [sample_pop[parents_id[0]],
               sample_pop[parents_id[1]]]
    return parents

## Recombinação dos pais

Os dois pais selecionados na função anterior são passados, juntamente com a dimensão N do tabuleiro. É então utilizado um método chamado de “Cut and crossover”, onde ambos os pais são “cortados” em um ponto aleatório e recombinados, com o Filho 1 sendo formado pela primeira metade do Pai 1 e a segunda metade do Pai 2, e o Filho 2 com a primeira metade do Pai 2 e a primeira metade do Pai 1.

Essa recombinação é feita com uma probabilidade de 100%, como recomendado no guia do trabalho.


In [65]:
def cut_and_crossfill(N, parents):
    cross_point = rd.randint(1, N-1)

    p1 = [parents[0][:cross_point] , parents[0][cross_point:]]
    p2 = [parents[1][:cross_point] , parents[1][cross_point:]]

    f1 = p1[0]
    f2 = p2[0]
    for i in range(len(p1[1])):
        f1.append(p2[1][i])
        f2.append(p1[1][i])

    return [f1, f2]

## Mutação dos filhos

Os filhos são posteriormente mutados por uma função que receberá as duas soluções geradas pela função anterior, uma “taxa de mutação” e o valor N. Com uma probabilidade igual a taxa de mutação passada, dois campos aleatórios do vetor de cada solução serão invertidos. O resultado final é retornado pela função.

Os valores empregados foram recomendados no guia do trabalho.


In [66]:
def mutate_offspring(offspring, mutation_rate, N):
    for of in offspring:
        if rd.random() < mutation_rate:
            id1 = rd.randint(0, N-1)
            id2 = rd.randint(0, N-1)
            aux = of[id1]
            of[id1] = of[id2]
            of[id2] = aux
    return offspring

## Cria e seleciona nova geração

A geração atual, juntamente com os filhos gerados, num total de “tamanho da população” + 2, são passados e analisados. São então selecionadas uma quantidade de soluções igual ao tamanho da população, descartando duas, formando a nova geração.

In [67]:
def select_new_generation(poulation, offspring):
    new_generation = poulation + offspring

    new_pop_fit = [0] * len(new_generation)
    for i in range(len(new_generation)):
        new_pop_fit[i] = fitness_nq(new_generation[i])

    new_pop_id = argsort(new_pop_fit)[:base_population]

    next_generation = []
    for i in range(len(new_generation)):
        if i in new_pop_id:
            next_generation.append(new_generation[i])

    return next_generation

## Desenha gráfico

Função de desenho do gráfico.

In [68]:
def draw_graph(datax, datay):
    plt.plot(range(datax+1), datay[0], "-g", label="Medium")
    plt.plot(range(datax+1), datay[1], "-r", label="Best")
    plt.legend(loc="upper right")
    plt.xlabel('Generation')
    plt.ylabel('Fittness')
    plt.show()

## Execução

In [71]:
N = 0
try:
    N = int(input())
except:
    raise ValueError('ERROR: o valor digitado não é valido!')

max_generation = 10 * N
print("Número de rainhas do problema: ", N)
print("Máximo de gerações: ", max_generation)
print()

pupulation = init_population(base_population, N)
population_fitt = [0] * base_population

for i in range(base_population):
    population_fitt[i] = fitness_nq(pupulation[i])
print("Fittness da população inicial:")
print(population_fitt)

count_gen = 0
datay = [[], []]
for i in range(max_generation):
    parents = select_parents(pupulation, pop_sample)

    offspring = cut_and_crossfill(N, parents)

    mutate_offspring(offspring, mutation_rate, N)

    pupulation = select_new_generation(pupulation, offspring)

    for j in range(base_population):
        population_fitt[j] = fitness_nq(pupulation[j])

    datay[0].append(sum(population_fitt) / len(population_fitt))
    datay[1].append(min(population_fitt))

    count_gen = i
    if verify_condition(population_fitt):
        break

print()
print("Geração final: ", (count_gen+1))
print()
print("Fittness da população final:")
print(population_fitt)
print()
print("Melhor indivíduo:")
best_pop = pupulation[get_best_solution(population_fitt)]
print(best_pop)
draw_pop(best_pop)

draw_graph(count_gen, datay)

Número de rainhas do problema:  8
Máximo de gerações:  80

Fittness da população inicial:
[20, 8, 8, 4, 10, 12, 8, 8, 20, 10, 16, 8, 10, 8, 8, 6, 4, 14, 8, 6]

Geração final:  49

Fittness da população final:
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 2, 2, 2, 0]

Melhor indivíduo:


KeyboardInterrupt: 